In [4]:
import numpy as np
import numpy_financial as npf
import pandas as pd

def bond_price(face_value, coupon_rate, periods, interest_rate):
    coupon = face_value * coupon_rate
    discount_factor = 1 / (1 + interest_rate)
    
    price = (coupon * (1 - discount_factor ** periods) / interest_rate) + (face_value * discount_factor ** periods)
    return price

def bond_yield(face_value, coupon_rate, periods, price):
    coupon = face_value * coupon_rate
    
    ytm = npf.irr([-price] + [coupon] * (periods - 1) + [face_value + coupon])
    return ytm

def bond_duration(face_value, coupon_rate, periods, interest_rate):
    coupon = face_value * coupon_rate
    discount_factor = 1 / (1 + interest_rate)
    
    weighted_cash_flows = [(t * coupon * discount_factor ** t) for t in range(1, periods + 1)]
    weighted_cash_flows.append(periods * face_value * discount_factor ** periods)
    
    price = bond_price(face_value, coupon_rate, periods, interest_rate)
    duration = sum(weighted_cash_flows) / price
    
    return duration

def bond_convexity(face_value, coupon_rate, periods, interest_rate):
    coupon = face_value * coupon_rate
    discount_factor = 1 / (1 + interest_rate)
    
    weighted_cash_flows = [(t * (t + 1) * coupon * discount_factor ** (t + 2)) for t in range(1, periods + 1)]
    weighted_cash_flows.append(periods * (periods + 1) * face_value * discount_factor ** (periods + 2))
    
    price = bond_price(face_value, coupon_rate, periods, interest_rate)
    convexity = sum(weighted_cash_flows) / (price * (1 + interest_rate) ** 2)
    
    return convexity

def modified_duration(duration, interest_rate):
    return duration / (1 + interest_rate)

def effective_duration(price_up, price_down, delta_rate, price):
    return (price_down - price_up) / (2 * price * delta_rate)

def effective_convexity(price_up, price_down, price, delta_rate):
    return (price_down + price_up - 2 * price) / (price * delta_rate ** 2)

def dollar_duration(price, modified_duration):
    return price * modified_duration

def credit_spread(risk_free_rate, yield_to_maturity):
    return yield_to_maturity - risk_free_rate

def adjusted_spread(market_yield, risk_free_rate):
    return market_yield - risk_free_rate

# Example usage
face_value = 1000  # Face value of the bond
coupon_rate = 0.05  # 5% annual coupon rate
periods = 10  # 10 years to maturity
interest_rates = [0.02, 0.03, 0.04, 0.05, 0.06, 0.07]  # Interest rate scenarios
delta_rate = 0.001  # Small change in interest rate for effective duration/convexity

# Additional inputs for market conditions
current_market_price = 950  # Current market price of the bond
market_yield = 0.052  # Market yield (5.2%)
risk_free_rate = 0.03  # Risk-free rate for credit and adjusted spread calculation

# Prepare the results in a list of dictionaries
results = []

for interest_rate in interest_rates:
    price = bond_price(face_value, coupon_rate, periods, interest_rate)
    ytm = bond_yield(face_value, coupon_rate, periods, price)
    duration = bond_duration(face_value, coupon_rate, periods, interest_rate)
    convexity = bond_convexity(face_value, coupon_rate, periods, interest_rate)
    mod_duration = modified_duration(duration, interest_rate)
    price_up = bond_price(face_value, coupon_rate, periods, interest_rate + delta_rate)
    price_down = bond_price(face_value, coupon_rate, periods, interest_rate - delta_rate)
    eff_duration = effective_duration(price_up, price_down, delta_rate, price)
    eff_convexity = effective_convexity(price_up, price_down, price, delta_rate)
    dollar_dur = dollar_duration(price, mod_duration)
    spread = credit_spread(risk_free_rate, ytm)
    adj_spread = adjusted_spread(market_yield, risk_free_rate)
    
    results.append({
        "Interest Rate": f"{interest_rate:.2%}",
        "Bond Price": f"${price:.2f}",
        "Yield to Maturity": f"{ytm:.2%}",
        "Macaulay Duration": f"{duration:.2f}",
        "Modified Duration": f"{mod_duration:.2f}",
        "Effective Duration": f"{eff_duration:.2f}",
          # Macaulay duration is the same as the standard duration
        "Convexity": f"{convexity:.2f}",
        "Effective Convexity": f"{eff_convexity:.2f}",
        "Dollar Duration": f"${dollar_dur:.2f}",
        "Credit Spread": f"{spread:.2%}",
        "Current Market Price": f"${current_market_price:.2f}",
        "Market Yield": f"{market_yield:.2%}",
        "Adjusted Spread": f"{adj_spread:.2%}"
    })

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Save the DataFrame to an Excel file without formatting
excel_filename = r'C:\Users\shrutika\Desktop\Shreyash\Bond Valuation\output.xlsx'
df.to_excel(excel_filename, index=False)

In [6]:
import numpy as np
import numpy_financial as npf
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, NamedStyle
from openpyxl.utils import get_column_letter

# Assuming the DataFrame 'df' has already been created and saved as an Excel file.



# Load the workbook and select the active worksheet
wb = load_workbook(excel_filename)
ws = wb.active

# Auto-adjust column widths
for col in ws.columns:
    max_length = 0
    column = col[0].column_letter  # Get the column letter
    for cell in col:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))
    ws.column_dimensions[column].width = max_length + 2  # Adjust for padding

# Make the first row bold and center-aligned
for cell in ws[1]:
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center", vertical="center")

# Define the red fill style
red_font = Font(color="FF0000")


# Apply the red negative style to all relevant cells in the DataFrame
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, max_col=ws.max_column):
    for cell in row:
        cell.alignment = Alignment(horizontal="center", vertical="center")
        if isinstance(cell.value, str) and '-' in cell.value:
            # Remove the '-' symbol
            cell_value = cell.value.replace('-', '')
            
            
            # Set the cell value and style
            cell.value = f'({cell_value})'
            cell.font = red_font

        # Ensure all cells are center-aligned
        cell.alignment = Alignment(horizontal="center", vertical="center")

# Save the formatted Excel file
wb.save(excel_filename)

print(f"Formatted results saved to {excel_filename}")


Formatted results saved to C:\Users\shrutika\Desktop\Shreyash\Bond Valuation\output.xlsx


In [1]:
def calculate_risk_metrics(face_value, coupon_rate, periods, interest_rates, current_price, risk_free_rate):
    results = []
    for rate in interest_rates:
        price = bond_price(face_value, coupon_rate, periods, rate)
        ytm = bond_yield(face_value, coupon_rate, periods, price)
        duration = bond_duration(face_value, coupon_rate, periods, rate)
        convexity = bond_convexity(face_value, coupon_rate, periods, rate)
        spread = credit_spread(risk_free_rate, ytm)
        
        risk_metrics = {
            "Interest Rate": rate,
            "Bond Price": price,
            "Yield to Maturity": ytm,
            "Duration": duration,
            "Convexity": convexity,
            "Credit Spread": spread
        }
        results.append(risk_metrics)
    
    return results



def normalize_metric(value, min_value, max_value):
    return (value - min_value) / (max_value - min_value)

def compute_weighted_risk_indicator(metrics, weights):
    normalized_metrics = {key: normalize_metric(value, min(values), max(values)) 
                           for key, value, values in metrics.items()}
    weighted_sum = sum(normalized_metrics[key] * weights.get(key, 1) for key in normalized_metrics)
    return weighted_sum
def risk_indicator(metrics_list, weights):
    indicators = [compute_weighted_risk_indicator(metrics, weights) for metrics in metrics_list]
    return indicators

import pandas as pd
import matplotlib.pyplot as plt

def generate_report(results):
    df = pd.DataFrame(results)
    df.to_excel('risk_indicator_report.xlsx', index=False)
    df.plot(x='Interest Rate', y='Risk Indicator', kind='line')
    plt.title('Risk Indicator vs Interest Rate')
    plt.xlabel('Interest Rate')
    plt.ylabel('Risk Indicator')
    plt.show()

In [11]:
import numpy as np
import numpy_financial as npf
from scipy.optimize import minimize

# Bond characteristics (example)
bonds = [
    {"face_value": 1000, "coupon_rate": 0.05, "periods": 10, "yield": 0.04},
    {"face_value": 1000, "coupon_rate": 0.04, "periods": 8, "yield": 0.035},
    {"face_value": 1000, "coupon_rate": 0.06, "periods": 12, "yield": 0.045},
    # Add more bonds as needed
]

# Target parameters
target_duration = 9  # Target portfolio duration
target_convexity = 40.0  # Target portfolio convexity
target_spread = 0.01  # Target portfolio credit spread
def bond_price(face_value, coupon_rate, periods, interest_rate):
    coupon = face_value * coupon_rate
    discount_factor = 1 / (1 + interest_rate)
    price = (coupon * (1 - discount_factor ** periods) / interest_rate) + (face_value * discount_factor ** periods)
    return price

def bond_duration(face_value, coupon_rate, periods, interest_rate):
    coupon = face_value * coupon_rate
    discount_factor = 1 / (1 + interest_rate)
    weighted_cash_flows = [(t * coupon * discount_factor ** t) for t in range(1, periods + 1)]
    weighted_cash_flows.append(periods * face_value * discount_factor ** periods)
    price = bond_price(face_value, coupon_rate, periods, interest_rate)
    duration = sum(weighted_cash_flows) / price
    return duration

def bond_convexity(face_value, coupon_rate, periods, interest_rate):
    coupon = face_value * coupon_rate
    discount_factor = 1 / (1 + interest_rate)
    weighted_cash_flows = [(t * (t + 1) * coupon * discount_factor ** (t + 2)) for t in range(1, periods + 1)]
    weighted_cash_flows.append(periods * (periods + 1) * face_value * discount_factor ** (periods + 2))
    price = bond_price(face_value, coupon_rate, periods, interest_rate)
    convexity = sum(weighted_cash_flows) / (price * (1 + interest_rate) ** 2)
    return convexity

def credit_spread(risk_free_rate, bond_yield):
    return bond_yield - risk_free_rate
def portfolio_metrics(weights, bonds, target_duration, target_convexity, target_spread, risk_free_rate=0.03):
    portfolio_duration = sum(weights[i] * bond_duration(bonds[i]['face_value'], bonds[i]['coupon_rate'], bonds[i]['periods'], bonds[i]['yield']) for i in range(len(bonds)))
    portfolio_convexity = sum(weights[i] * bond_convexity(bonds[i]['face_value'], bonds[i]['coupon_rate'], bonds[i]['periods'], bonds[i]['yield']) for i in range(len(bonds)))
    portfolio_spread = sum(weights[i] * credit_spread(risk_free_rate, bonds[i]['yield']) for i in range(len(bonds)))
    
    # Calculate the difference from target metrics
    duration_diff = ((portfolio_duration - target_duration)/target_duration) ** 2
    convexity_diff = ((portfolio_convexity - target_convexity)/target_convexity) ** 2
    spread_diff = ((portfolio_spread - target_spread)/target_spread) ** 2
    
    # Objective function to minimize
    return duration_diff + convexity_diff + spread_diff
# Initial guess (equal weighting)
initial_weights = np.array([1 / len(bonds)] * len(bonds))

# Set minimum and maximum weight constraints for each bond
min_weight = 0.05  # Minimum weight for any bond
max_weight = 0.50  # Maximum weight for any bond

# Define bounds for each bond
bounds = [(min_weight, max_weight) for _ in range(len(bonds))]

# Constraints: weights must sum to 1
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# Optimize the portfolio with the additional constraints
result = minimize(portfolio_metrics, initial_weights, args=(bonds, target_duration, target_convexity, target_spread), bounds=bounds, constraints=constraints)

# Optimal weights rounded to two decimal places
optimal_weights = np.round(result.x, 2)

# Output the results
print("Optimal Weights:", optimal_weights)

# Calculate the final portfolio metrics using the rounded weights
portfolio_duration = sum(optimal_weights[i] * bond_duration(bonds[i]['face_value'], bonds[i]['coupon_rate'], bonds[i]['periods'], bonds[i]['yield']) for i in range(len(bonds)))
portfolio_convexity = sum(optimal_weights[i] * bond_convexity(bonds[i]['face_value'], bonds[i]['coupon_rate'], bonds[i]['periods'], bonds[i]['yield']) for i in range(len(bonds)))
portfolio_spread = sum(optimal_weights[i] * credit_spread(0.03, bonds[i]['yield']) for i in range(len(bonds)))

print(f"Portfolio Duration: {portfolio_duration:.2f}")
print(f"Portfolio Convexity: {portfolio_convexity:.2f}")
print(f"Portfolio Spread: {portfolio_spread:.4f}")



Optimal Weights: [0.45 0.5  0.05]
Portfolio Duration: 7.65
Portfolio Convexity: 63.01
Portfolio Spread: 0.0078


In [3]:
pip install scipy

   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/44.5 MB ? eta -:--:--
   ---------------------------------------- 0.5/44.5 MB 1.7 MB/s eta 0:00:27
    --------------------------------------- 0.8/44.5 MB 1.5 MB/s eta 0:00:30
    --------------------------------------- 1.0/44.5 MB 1.1 MB/s eta 0:00:41
   - -------------------------------------- 1.3/44.5 MB 1.1 MB/s eta 0:00:40
   - -------------------------------------- 1.3/44.5 MB 1.1 MB/s eta 0:00:40
   - -------------------------------------- 1.3/44.5 MB 1.1 MB/s eta 0:00:40
   - -------------------------------------- 1.3/44.5 MB 1.1 MB/s eta 0:00:40
   - -------------------------------------- 1.3/44.5 MB 1.1 MB/s eta 0:00:40
   - -------------------------------------- 1.3/44.5 MB 1.1 MB/s eta 0:00:40
   - ----------------------